In [1]:
import pandas as pd

In [2]:
trail2data = pd.read_csv('trial2_data.csv', low_memory=False)

In [3]:
patients = pd.read_csv('data/Train/Patient_Profile.csv', low_memory=False)

In [7]:
patients.columns

Index(['Patient_ID', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
       'Facebook_Shared', 'Income', 'Education_Score', 'Age',
       'First_Interaction', 'City_Type', 'Employer_Category'],
      dtype='object')

In [8]:
patients_cols = ['Patient_ID', 'First_Interaction']
target_data = pd.merge(trail2data, patients[patients_cols], on='Patient_ID', how='left')

In [10]:
target_data.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,target,day,month,...,Category2_A,Category2_B,Category2_C,Category2_D,Category2_E,Category2_F,Category2_G,Category3_1,Category3_2,First_Interaction
0,489652,6578,4,0,0,0,2,1.0,10,9,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,06-Dec-04
1,507246,6578,45,5,0,0,7,0.0,18,8,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,08-Sep-04
2,491371,6578,0,0,0,0,0,0.0,24,7,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,04-Mar-03
3,485995,6578,0,0,0,0,0,1.0,1,10,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,30-Sep-05
4,511519,6578,0,0,0,0,0,1.0,26,9,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,19-Sep-05


In [15]:
original_data = pd.read_csv('data/Train/Train.csv', usecols=['Patient_ID','Registration_Date'])

In [39]:
reg_date_target_data = pd.merge(target_data, original_data, on='Patient_ID', how='outer')

In [41]:
reg_date = pd.to_datetime(reg_date_target_data.Registration_Date, infer_datetime_format=True)

In [42]:
first_interaction = pd.to_datetime(reg_date_target_data.First_Interaction, infer_datetime_format=True)

In [43]:
def calculate_first_interaction_diff(x):
    try:
        return x.days
    except AttributeError:
        return 0

reg_date_target_data['reg_diff'] = (reg_date - first_interaction).apply(calculate_first_interaction_diff)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f2780c7a5c0>>
Traceback (most recent call last):
  File "/home/jayant/anaconda3/lib/python3.5/site-packages/xgboost/core.py", line 287, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


In [44]:
reg_date_target_data.columns

Index(['Patient_ID', 'Health_Camp_ID', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5',
       'target', 'day', 'month', 'year', 'days_active', 'Category1_First',
       'Category1_Second', 'Category1_Third', 'Category2_A', 'Category2_B',
       'Category2_C', 'Category2_D', 'Category2_E', 'Category2_F',
       'Category2_G', 'Category3_1', 'Category3_2', 'First_Interaction',
       'Registration_Date', 'reg_diff'],
      dtype='object')

In [45]:
del reg_date_target_data['Registration_Date']
del reg_date_target_data['First_Interaction']

In [46]:
trainset = reg_date_target_data[0:75278]
testset = reg_date_target_data[75278:]

In [47]:
import xgboost as xgb

In [48]:
partial_trainset = trainset[trainset['year'].isin([2004, 20005])]
validation_set = trainset[trainset['year'].isin([2006])]

train_cols = [col for col in trainset.columns if col not in ['target', 'Patient_ID', 'Health_Camp_ID']]
X_train = partial_trainset[train_cols]
Y_train = partial_trainset['target']

X_val = validation_set[train_cols]
Y_val = validation_set['target']

X_test = testset[train_cols]
Y_test = testset['target']

dtrain = xgb.DMatrix(X_train.as_matrix(), label=Y_train.as_matrix(), feature_names=X_train.columns)
dtest = xgb.DMatrix(X_test.as_matrix(), label=Y_test.as_matrix(), feature_names=X_test.columns)
dval = xgb.DMatrix(X_val.as_matrix(), label=Y_val.as_matrix(), feature_names=X_val.columns)

watchlist = [(dtrain, 'train'), (dval, 'eval')]

In [49]:
params = {}
params["objective"] = "binary:logistic"
params["eta"] = 0.003
params["min_child_weight"] = 1
params["subsample"] = 0.8
# params["colsample_bytree"] = 0.8
params["scale_pos_weight"] = 1.0
# params["silent"] = 1
params["max_depth"] = 5
params["nthread"] = 4
params["eval_metric"] = "auc"

In [50]:
num_rounds = 500
model = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	train-auc:0.702557	eval-auc:0.599729
[1]	train-auc:0.731417	eval-auc:0.574681
[2]	train-auc:0.739621	eval-auc:0.579001
[3]	train-auc:0.739055	eval-auc:0.573954
[4]	train-auc:0.739660	eval-auc:0.575863
[5]	train-auc:0.741077	eval-auc:0.578741
[6]	train-auc:0.742555	eval-auc:0.576181
[7]	train-auc:0.745703	eval-auc:0.574912
[8]	train-auc:0.746496	eval-auc:0.574085
[9]	train-auc:0.742330	eval-auc:0.576050
[10]	train-auc:0.730991	eval-auc:0.580234
[11]	train-auc:0.736514	eval-auc:0.581797
[12]	train-auc:0.747098	eval-auc:0.573623
[13]	train-auc:0.749087	eval-auc:0.568292
[14]	train-auc:0.748930	eval-auc:0.569806
[15]	train-auc:0.735445	eval-auc:0.579320
[16]	train-auc:0.735151	eval-auc:0.578704
[17]	train-auc:0.735517	eval-auc:0.575988
[18]	train-auc:0.736432	eval-auc:0.576191
[19]	train-auc:0.736144	eval-auc:0.576241
[20]	train-auc:0.738839	eval-auc:0.570461
[21]	train-auc:0.735705	eval-auc:0.576071
[22]	train-auc:0.737654	eval-auc:0.573959
[23]	train-auc:0.739924	eval-auc:0.565240
[2